 **В этой тетрадке только код, который создает базу данных. Дальше в коллабе стало совсем не удобно работать, поэтому я перешла в пайчарм.**


##Обработка данных

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('output_f_2.csv')
#print(df['full_text'].isnull().sum())
df['full_text'].dropna()
df['full_text'] = df['full_text'].fillna('')
df['full_text'] = df['full_text'].astype(str)
df = df.drop(index=1)

In [ ]:
df = df.drop(index=0)

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head(10)

,href,title,pub_day,full_text,tags
0,https://peopletalk.ru/article/rezhissera-gladi...,Режиссера «Гладиатора 2» назвали ленивым и нет...,29 ноября,Оператор фильма «Гладиатор 2» Джон Мэтисон рас...,"['Гладиатор 2', 'Ридли Скотт']"
1,https://peopletalk.ru/article/redkie-foto-emma...,Редкие фото: Эмма Робертс показала подросшего ...,29 ноября,33-летняя Эмма Робертс не так часто делится с ...,"['дети звезд', 'Эмма Робертс']"
2,https://peopletalk.ru/article/beremennaya-zhiz...,"Беременная Жизель Бундхен показала, как провел...",29 ноября,Беременная Жизель Бундхен в этом году забрала ...,"['дети звезд', 'Жизель Бундхен']"
3,https://peopletalk.ru/article/sledy-ostavlenny...,"Следы, оставленные 1,5 млн лет назад в Африке,...",29 ноября,"Согласно исследованию, два вида древних людей,...","['археологи', 'Исследование', 'ученые']"
4,https://peopletalk.ru/article/den-narodnogo-ed...,День народного единства: все о празднике,04 ноября,"Сегодня, 4 ноября, в России отмечают День наро...",['праздник']
5,https://peopletalk.ru/article/shou-simpsony-sp...,Шоу «Симпсоны» спустя 35 лет покидает «голос» ...,21 ноября,Сериал «Симпсоны» длится десятилетиями и кажет...,['Симпсоны']
6,https://peopletalk.ru/article/pol-meskal-shoki...,Пол Мескал шокирован после встречи с победител...,21 ноября,Фанату Пола Мескала повезло увидеться со своим...,['Пол Мескал']
7,https://peopletalk.ru/article/pi-piddi-v-tyurm...,Пи Пидди в тюрьме: хронология скандала с рэпер...,26 сентября,Вокруг ареста Пи Пидди – Шона Комбса (настояще...,"['арест', 'домогательства', 'обвинения', 'Пи Д..."
8,https://peopletalk.ru/article/proshhaj-legenda...,"Прощай, легенда: создатель книжного портала «Ф...",24 сентября,"Еще одна ниточка, соединяющая молодость миллен...","['книги', 'миллениалы']"
9,https://peopletalk.ru/article/anna-hilkevich-p...,Анна Хилькевич провела социальный эксперимент ...,29 ноября 2023,Не так давно звезда «Универа» явилась на преми...,"['Анна Хилькевич', 'пластика ягодиц', 'Хилькев..."


In [ ]:
def remove_empty_strings(df, column):  # удаляю строки с пустыми тэгами, если такие есть
    return df[df[column] != '[]']

df = remove_empty_strings(df, 'tags')

In [ ]:
df['tags'][0]

"['Гладиатор 2', 'Ридли Скотт']"

In [ ]:
import ast
df['tags'] = df['tags'].apply(ast.literal_eval)

In [ ]:
df['tags'][0]

['Гладиатор 2', 'Ридли Скотт']

In [ ]:
print(df.isnull().sum())

href         0
title        0
pub_day      0
full_text    0
tags         0
dtype: int64


In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1191 entries, 0 to 1217
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   href       1191 non-null   object
 1   title      1191 non-null   object
 2   pub_day    1191 non-null   object
 3   full_text  1191 non-null   object
 4   tags       1191 non-null   object
dtypes: object(5)
memory usage: 88.1+ KB
None


## Создание базы данных

In [ ]:
import sqlite3

In [ ]:

# Подключение к базе данных
conn = sqlite3.connect('articles.db')
cur = conn.cursor()

# Создание главной таблицы articles
cur.execute("""
CREATE TABLE IF NOT EXISTS articles (
    article_id INTEGER PRIMARY KEY AUTOINCREMENT,
    href TEXT NOT NULL UNIQUE,
    title TEXT NOT NULL,
    full_text TEXT NOT NULL
)
""")

# Создание таблицы publication
cur.execute("""
CREATE TABLE IF NOT EXISTS publication (
    publication_id INTEGER PRIMARY KEY AUTOINCREMENT,
    pub_day TEXT NOT NULL UNIQUE
)
""")

# Создание таблицы article_publication
cur.execute("""
CREATE TABLE IF NOT EXISTS article_publication (
    article_id INTEGER,
    publication_id INTEGER,
    FOREIGN KEY (article_id) REFERENCES articles (article_id),
    FOREIGN KEY (publication_id) REFERENCES publication (publication_id),
    PRIMARY KEY (article_id, publication_id)
)
""")

# Создание таблицы tags
cur.execute("""
CREATE TABLE IF NOT EXISTS tags (
    tag_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL UNIQUE
)
""")

# Создание таблицы article_tags
cur.execute("""
CREATE TABLE IF NOT EXISTS article_tags (
    article_id INTEGER,
    tag_id INTEGER,
    FOREIGN KEY (article_id) REFERENCES articles (article_id),
    FOREIGN KEY (tag_id) REFERENCES tags (tag_id),
    PRIMARY KEY (article_id, tag_id)
)
""")



In [ ]:
conn.commit()
conn.close()